In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3
import pickle
import re

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from lxml import etree
from collections import defaultdict
from pprint import pprint
from config.assistant import *
from config.site_config import *
from tqdm import tqdm

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

request_headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36'),
}

In [2]:
job_group = {}

In [3]:
def save_dictionary(job_company, company_link, job_posting, posting_link):
    global job_group
    
    # 신규 회사명 인입
    if not job_company in job_group:    
        job_group[job_company] = []
    
    for company in job_group[job_company]:
        if company['title'] == job_posting:    # 채용공고명 - 중복
            break
    else:
        job_group[job_company].append({'company': job_company, \
                                       'company_link': company_link, \
                                       'title': job_posting, \
                                       'title_link': posting_link, \
                                       'title_idx': len(job_group[job_company]), \
                                       'input_date': datetime.date.today().isoformat(), \
                                       'status': 'wait'
                                      })
    return None

In [4]:
def filtering_company_name(job_company):
    
    # 회사명 - ()괄호 안의 모든 단어 제거
    company_re = re.search('\(.*\)', job_company)
    if company_re:
        str_filtering = job_company[company_re.span()[0]:company_re.span()[1]]
        job_company = job_company.replace(str_filtering, '')
        
    # 회사명 - 특정 단어 제거
    for remove_word in remove_words:
        if remove_word in job_company:
            job_company = job_company.replace(remove_word, '')
    
    # 회사명 - 완전히 일치하는 회사명 리스트 필터링
    if job_company in filter_list_company_name:
        return 0, "none"
    
    # 회사명 - 특정 단어가 포함되어 있는 회사 필터링
    for remove_word in filter_list_company_name_part:
        if remove_word in job_company:
            return 0, "none"

    return 1, job_company

### 사람인

In [5]:
tic()

for search_word in search_words:

    item_count = 0
    current_page = 1
    search_result_num = 0

    # =======================================================================================
    
    # 검색 결과 리스트 페이지 별 확인
    
    search_loop = True
    while True:
        search_link = f'https://www.saramin.co.kr/zf_user/search/recruit'\
            + f'?search_area=main'\
            + f'&search_done=y'\
            + f'&search_optional_item=n'\
            + f'&searchType=search'\
            + f'&recruitSort=relation'\
            + f'&searchword={search_word}'\
            + f'&recruitPage={current_page}'\
            + f'&recruitPageCount={page_view_items}'\
            + f'&company_cd={company_cd}'\
            + f'&mainSearch=y'
        response = requests.get(search_link, headers=request_headers, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        dom = etree.HTML(str(soup))
        
        # =======================================================================================
        
        # 검색단어 리스트 별 검색 결과 개수, 검색 결과가 없으면 종료
        
        try:
            num_str = dom.xpath('//*[@id="recruit_info"]/div[1]/span')[0].text
            search_result_num = int(re.sub(r'[^0-9]', '', num_str))
        except:
            break
        finally:
            if current_page == 1:
                print(f'{search_word}: {search_result_num}')

        # =======================================================================================
        
        current_page += 1

        
        page_items = len(soup.find_all('h2', attrs={'class': 'job_tit'}))

        # 해당 페이지 리스트 (1 ~ 100)
        for i in range(1, page_items+1):
            elements = soup.select(f'div.content > div:nth-child({i})')[0]

            # 회사명
            job_company = elements.find('a', attrs={'class': 'track_event data_layer'}).text
            filtering_result = filtering_company_name(job_company)
            if filtering_result[0] == 0:
                continue
            else:
                job_company = filtering_result[1]

            # 채용공고명
            job_posting = elements.find('a', attrs={'class': 'data_layer'})['title']

            # 채용공고 링크
            posting_link = base_link_saram + elements.find('a', attrs={'class': 'data_layer'})['href']

            # 회사 링크
            company_link = base_link_saram + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

            # Dictionary에 저장
            save_dictionary(job_company, company_link, job_posting, posting_link)

            item_count += 1

print("------------")
toc()

IMAGE RECOGNITION: 19
------------
Elapsed time is 28.312422513961792 seconds.


In [6]:
len(job_group)

14

In [7]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

19


### 잡코리아

In [6]:
# 잡코리아
# 저장된 키워드 별 검색

tic()
for search_word in search_words:

    item_count = 0
    current_page = 1
    
    # 검색 결과 리스트 페이지 별 확인
    search_loop = True
    while(search_loop):
        search_link = f'https://www.jobkorea.co.kr/Search/' \
                    +f'?stext={search_word}' \
                    +f'&tabType=recruit&Page_No={current_page}'

        response = requests.get(search_link, headers=request_headers, verify=False)
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        


        page_items = len(soup.find_all('a', attrs={'class': 'title dev_view'}))
        
        if page_items == 0:
            search_loop = False
        else:
            for i in range(0, page_items):
                # 회사명
                job_company = soup.find_all('a', attrs={'class': 'name dev_view'})[i].text
                filtering_result = filtering_company_name(job_company)
                if filtering_result[0] == 0:
                    continue
                else:
                    job_company = filtering_result[1]
                
                # 회사 링크
                company_link = base_link_korea + soup.find_all('a', attrs={'class': 'name dev_view'})[i]['href']
                
                # 채용공고
                job_posting = soup.find_all('a', attrs={'class': 'title dev_view'})[i].text
                job_posting = job_posting.replace('\r\n', '')
                job_posting = job_posting.strip()

                # 채용공고 링크
                posting_link = base_link_korea + soup.find_all('a', attrs={'class': 'title dev_view'})[i]['href']

                # Dictionary에 저장
                save_dictionary(job_company, company_link, job_posting, posting_link)
                
                item_count += 1
        current_page += 1
    print(f'{search_word}: {item_count}')
toc()

컴퓨터비전: 1105
COMPUTER VISION: 110
영상처리: 1087
IMAGE PROCESSING: 58
딥러닝: 1154
DEEP LEARNING: 188
머신러닝: 1750
MACHINE LEARNING: 1754
이미지인식: 129
IMAGE RECOGNITION: 22
이미지분석: 2220
IMAGE ANALYSIS: 23
VISION AI: 254
OPENCV: 144
Elapsed time is 537.1458041667938 seconds.


In [7]:
len(job_group)

5688

In [8]:
nCnt = 0
for i in job_group:
    for j in job_group[i]:
        nCnt += 1
print(nCnt)

12694


In [9]:
# JSON 파일 쓰기
tf = open(file_path, 'w')
json.dump(job_group, tf)
tf.close()

In [5]:
# JSON 파일 불러오기
with open(file_path, 'r') as file:
    job_group = json.load(file)
print(len(job_group))

5494
